## Imports

In [14]:
import hydra
from omegaconf import DictConfig, OmegaConf
import logging
import wandb
import os

# Import boilerplate dependencies from your training framework
from nn_core.common.utils import enforce_tags, seed_index_everything
from nn_core.callbacks import NNTemplateCore
from nn_core.common import PROJECT_ROOT
from nn_core.model_logging import NNLogger

# Import the functions from your modified files.
from mass.clip_interpret.compute_complete_text_set import run_completeness
from mass.clip_interpret.compute_text_set_projection import run_text_features

import time
import numpy as np
import torch
import os
import einops
import tqdm
import open_clip
import wandb
import logging
from pathlib import Path

from mass.utils.io_utils import load_model_from_disk
from mass.utils.utils import compute_task_dict
from mass.task_vectors.task_singular_vectors import get_svd_dict

pylogger = logging.getLogger(__name__)

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import hydra
from hydra import initialize, compose
from typing import Dict, List

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path=str("../conf"), job_name="clip_interpret")
cfg = compose(config_name="clip_interpret")

In [17]:
def boilerplate(cfg: DictConfig):
    """
    Full boilerplate initialization mimicking your larger script.
    This sets up tags, restores any checkpoints, and initializes the advanced logger.
    """
    cfg.core.tags.append(f"clip_interpret")

    # Initialize template core for resuming experiments
    template_core = NNTemplateCore(restore_cfg=cfg.train.get("restore", None))
    # Initialize the NNLogger with advanced logging configuration
    logger = NNLogger(
        logging_cfg=cfg.train.logging, cfg=cfg, resume_id=template_core.resume_id
    )
    logger.upload_source()
    return logger, template_core

In [18]:
# Initialize boilerplate (advanced logging, tags, etc.)
logger, template_core = boilerplate(cfg)

2025-03-17 12:02:34 INFO     Restoring with mode: <None>                                         ]8;id=848647;file:///media/donato/Extra-storage/Code/model-merging/mass/.venv/lib/python3.11/site-packages/nn_core/resume.py\nn_core.resume]8;;\:]8;id=770386;file:///media/donato/Extra-storage/Code/model-merging/mass/.venv/lib/python3.11/site-packages/nn_core/resume.py#122\122]8;;\

                    INFO     Instantiating <WandbLogger>                                   ]8;id=894215;file:///media/donato/Extra-storage/Code/model-merging/mass/.venv/lib/python3.11/site-packages/nn_core/model_logging.py\nn_core.model_logging]8;;\:]8;id=161055;file:///media/donato/Extra-storage/Code/model-merging/mass/.venv/lib/python3.11/site-packages/nn_core/model_logging.py#41\41]8;;\

/media/donato/Extra-storage/Code/model-merging/mass/.venv/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


                    INFO     Uploading source code to W&B                                  ]8;id=880152;file:///media/donato/Extra-storage/Code/model-merging/mass/.venv/lib/python3.11/site-packages/nn_core/model_logging.py\nn_core.model_logging]8;;\:]8;id=841803;file:///media/donato/Extra-storage/Code/model-merging/mass/.venv/lib/python3.11/site-packages/nn_core/model_logging.py#63\63]8;;\

In [19]:
name = cfg.text_descriptions.replace(".txt", "")
output_path = os.path.join(cfg.misc.output_dir, f"{name}_{cfg.model}.npy")

if os.path.exists(output_path):
    pylogger.info(
        f"Output file already exists: {output_path}. Skipping computation."
    )
else:
    pylogger.info("Running text feature extraction...")
    run_text_features(cfg)

2025-03-17 12:02:42 INFO     Output file already exists:                                                 ]8;id=616709;file:///tmp/ipykernel_108035/4104344879.py\__main__]8;;\:]8;id=151195;file:///tmp/ipykernel_108035/4104344879.py#5\5]8;;\
                             /media/donato/Extra-storage/Code/model-merging/mass/results/interpret/ViT-B           
                             -16/image_descriptions_general_ViT-B-16.npy. Skipping computation.                    

In [20]:
zeroshot_encoder_statedict = load_model_from_disk(cfg.misc.pretrained_checkpoint)

finetuned_name = (
    lambda name: Path(cfg.misc.ckpt_path) / f"{name}Val" / "nonlinear_finetuned.pt"
)
finetuned_models = {
    dataset: load_model_from_disk(finetuned_name(dataset))
    for dataset in cfg.task_vectors.to_apply
}

task_dicts = {}
for dataset in cfg.task_vectors.to_apply:
    task_dicts[dataset] = compute_task_dict(
        zeroshot_encoder_statedict, finetuned_models[dataset]
    )
    del finetuned_models[dataset]  # Delete one model at a time
    torch.cuda.empty_cache()

svd_dict = get_svd_dict(
    task_dicts, cfg.eval_datasets, cfg.misc.svd_path, cfg.svd_compress_factor
)

model, _, preprocess = open_clip.create_model_and_transforms(
    cfg.model, pretrained="openai", cache_dir=cfg.misc.openclip_cachedir
)
model.to(cfg.device)
all_images = set()

# Load text features from file
name = cfg.text_descriptions.replace(".txt", "")
text_features_path = os.path.join(cfg.misc.output_dir, f"{name}_{cfg.model}.npy")

with open(text_features_path, "rb") as f:
    text_features = np.load(f)
pylogger.info(f"Loaded text features from {text_features_path}")

# Load text descriptions (each line is one text)
text_file = os.path.join(cfg.misc.description_dir, f"{cfg.text_descriptions}")
with open(text_file, "r") as f:
    lines = [line.strip() for line in f.readlines()]
pylogger.info(f"Loaded text descriptions from {text_file}")

2025-03-17 12:02:53 INFO     Using compression ratio: 0.0500            ]8;id=216323;file:///media/donato/Extra-storage/Code/model-merging/mass/src/mass/task_vectors/task_singular_vectors.py\mass.task_vectors.task_singular_vectors]8;;\:]8;id=659968;file:///media/donato/Extra-storage/Code/model-merging/mass/src/mass/task_vectors/task_singular_vectors.py#292\292]8;;\

                    INFO     Loading precomputed SVD dictionary from:   ]8;id=18383;file:///media/donato/Extra-storage/Code/model-merging/mass/src/mass/task_vectors/task_singular_vectors.py\mass.task_vectors.task_singular_vectors]8;;\:]8;id=976865;file:///media/donato/Extra-storage/Code/model-merging/mass/src/mass/task_vectors/task_singular_vectors.py#299\299]8;;\
                             /media/donato/Extra-storage/Code/model-mer                                            
                             ging/mass/checkpoints/svd_dict_ViT-B-16_co                                            
                             mpress_20.pt                                                                          

2025-03-17 12:02:54 INFO     Loading pretrained ViT-B-16 from OpenAI.                                       ]8;id=864248;file:///media/donato/Extra-storage/Code/model-merging/mass/.venv/lib/python3.11/site-packages/open_clip/factory.py\root]8;;\:]8;id=471635;file:///media/donato/Extra-storage/Code/model-merging/mass/.venv/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

2025-03-17 12:02:55 INFO     Loaded text features from                                                  ]8;id=686208;file:///tmp/ipykernel_108035/3989036501.py\__main__]8;;\:]8;id=941452;file:///tmp/ipykernel_108035/3989036501.py#35\35]8;;\
                             /media/donato/Extra-storage/Code/model-merging/mass/results/interpret/ViT-            
                             B-16/image_descriptions_general_ViT-B-16.npy                                          

                    INFO     Loaded text descriptions from                                              ]8;id=463677;file:///tmp/ipykernel_108035/3989036501.py\__main__]8;;\:]8;id=436979;file:///tmp/ipykernel_108035/3989036501.py#41\41]8;;\
                             /media/donato/Extra-storage/Code/model-merging/mass/misc/text_descriptions            
                             /image_descriptions_general.txt                                                       

In [21]:

# Prepare a wandb Table for fancy terminal logging
results_table = wandb.Table(columns=["Task", "Top Texts"])

# Optionally also write the results to a file
output_file = os.path.join(
    cfg.misc.output_dir,
    f"{cfg.dataset}_completeness_{cfg.text_descriptions}_top_{cfg.texts_per_task}_heads_{cfg.model}.txt",
)

## TextSpan

In [22]:
@torch.no_grad()
def replace_with_iterative_removal(data, text_features, texts, iters, device):
    
    results = []
    vh = data  # in our case we already have "vectors"...
    text_features = (
        vh.T.dot(np.linalg.inv(vh.dot(vh.T)).dot(vh)).dot(text_features.T).T
    )  # Project the text to the span of W_OV
    
    data = torch.from_numpy(data).float().to(device)
    mean_data = data.mean(dim=0, keepdim=True)
    data = data - mean_data
    
    reconstruct = einops.repeat(mean_data, "A B -> (C A) B", C=data.shape[0])
    reconstruct = reconstruct.detach().cpu().numpy()
    text_features = torch.from_numpy(text_features).float().to(device)

    for i in range(iters):

        projection = data @ text_features.T
        projection_std = projection.std(axis=0).detach().cpu().numpy()
        top_n = np.argmax(projection_std)
        results.append(texts[top_n])
        text_norm = text_features[top_n] @ text_features[top_n].T
        reconstruct += (
            (
                (data @ text_features[top_n] / text_norm)[:, np.newaxis]
                * text_features[top_n][np.newaxis, :]
            )
            .detach()
            .cpu()
            .numpy()
        )
        data = data - (
            (data @ text_features[top_n] / text_norm)[:, np.newaxis]
            * text_features[top_n][np.newaxis, :]
        )
        text_features = (
            text_features
            - (text_features @ text_features[top_n] / text_norm)[:, np.newaxis]
            * text_features[top_n][np.newaxis, :]
        )
    return reconstruct, results

In [23]:
output_str = ''

for task in svd_dict.keys():
    
    pylogger.info(f"Processing Task: {task}")

    output_str += f"------------------\n"
    output_str += f"V for task {task}\n"
    output_str += f"------------------\n"

    # Retrieve SVD components for the current task
    u = svd_dict[task][cfg.layer]["u"].to(cfg.device)
    s = torch.diag_embed(svd_dict[task][cfg.layer]["s"]).to(cfg.device)
    v = svd_dict[task][cfg.layer]["v"].to(cfg.device)
    # pylogger.info(f"v shape for task {task}: {v.shape}")

    # Compute the projected matrix
    v_proj = s @ v @ model.visual.proj

    # Apply the iterative removal procedure
    reconstruct, images = replace_with_iterative_removal(
        v_proj.detach().cpu().numpy(),
        text_features,
        lines,
        cfg.texts_per_task,
        cfg.device,
    )

    all_images |= set(images)
    for text in images:
        output_str += f"{text}\n"
    results_table.add_data(task, "\n".join(images))
    pylogger.info(f"Task {task}: {images}")

    break 

print(output_str)

                    INFO     Processing Task: Cars                                                       ]8;id=677806;file:///tmp/ipykernel_108035/743796930.py\__main__]8;;\:]8;id=93015;file:///tmp/ipykernel_108035/743796930.py#5\5]8;;\

                    INFO     Task Cars: ['Image of a car', 'Nostalgic expressions', 'Artwork featuring  ]8;id=906931;file:///tmp/ipykernel_108035/743796930.py\__main__]8;;\:]8;id=148371;file:///tmp/ipykernel_108035/743796930.py#33\33]8;;\
                             Morse code typography', 'Image of a delivery van', 'Image snapped in the              
                             Swiss chocolate factories']                                                           

------------------
V for task Cars
------------------
Image of a car
Nostalgic expressions
Artwork featuring Morse code typography
Image of a delivery van
Image snapped in the Swiss chocolate factories



## Nearest-neighbor

In [24]:
def get_nearest_text_descriptions(data, text_features, texts, device='cuda'):
    results = []
    vh = data  # in our case we already have "vectors"...
    text_features = (
        vh.T.dot(np.linalg.inv(vh.dot(vh.T)).dot(vh)).dot(text_features.T).T
    )  # Project the text to the span of W_OV
    
    data = torch.from_numpy(data).float().to(device)
    
    # mean_data = data.mean(dim=0, keepdim=True)
    # data = data - mean_data
    
    text_features = torch.from_numpy(text_features).float().to(device)

    projection = data @ text_features.T

    # take the top 5 texts
    top_1 = torch.argmax(projection)
    result = texts[top_1]

    return result


In [25]:
output_str = ''

for task in svd_dict.keys():
    
    pylogger.info(f"Processing Task: {task}")

    output_str += f"------------------\n"
    output_str += f"V for task {task}\n"
    output_str += f"------------------\n"

    # Retrieve SVD components for the current task
    u = svd_dict[task][cfg.layer]["u"].to(cfg.device)
    s = torch.diag_embed(svd_dict[task][cfg.layer]["s"]).to(cfg.device)
    v = svd_dict[task][cfg.layer]["v"].to(cfg.device)
    # pylogger.info(f"v shape for task {task}: {v.shape}")

    # Compute the projected matrix
    # shall we have s here?
    # v_proj = s @ v @ model.visual.proj
    v_proj = s @ v @ model.visual.proj

    all_captions = []

    for v in v_proj:
        # Apply the iterative removal procedure
        captions = get_nearest_text_descriptions(
            v.unsqueeze(0).detach().cpu().numpy(),
            text_features,
            lines,
            cfg.device,
        )

        all_captions.append(captions)

    pylogger.info(f"Task {task}: {all_captions}")

    for text in all_captions:
        output_str += f"{text}\n"


                    INFO     Processing Task: Cars                                                       ]8;id=727797;file:///tmp/ipykernel_108035/4266128688.py\__main__]8;;\:]8;id=986624;file:///tmp/ipykernel_108035/4266128688.py#5\5]8;;\

                    INFO     Task Cars: ['Photograph taken in a car', 'Image snapped in the Swiss       ]8;id=471171;file:///tmp/ipykernel_108035/4266128688.py\__main__]8;;\:]8;id=699095;file:///tmp/ipykernel_108035/4266128688.py#35\35]8;;\
                             chocolate factories', 'Image snapped in the Swiss chocolate factories',               
                             'Photo taken in Great Barrier Reef', 'Caricature of an iconic explorer',              
                             'Illustration of a hidden enchanted castle', 'Photograph taken in a retro             
                             diner', 'A glove', 'Detailed illustration of a futuristic quantum realm',             
                             'Vintage nostalgia', 'Ikat design', 'Close-up of a textured ceramic',                 
                             'Mysterious day scene', 'Nostalgic city pulse', 'Artwork featuring Morse              
                             code typography', 'a photo of cardiff', 'Image with holographic retro                 
                             arcade aesthetics', 'Image of a delivery van', 'Picture taken in the                  
                             Spanish Flamenco festivals', 'Detailed illustration of a vehicle', 'A                 
                             magnolia', 'Nostalgic city lights', 'A geyser', 'A trunk (of a tree or an             
                             elephant)', 'Desert sand dunes', 'Impressionist portrait painting', 'Photo            
                             taken in Monument Valley', 'Aerial view of a vineyard', 'Picture captured             
                             in the New York skyline', 'Image of a bus', 'Serene countryside sunrise',             
                             'A cracked surface texture', 'Image with quilted patchwork design', 'Photo            
                             taken in the Brazilian carnival', 'fleeting soap bubble art', 'Aerial view            
                             of a serene meadow', 'A traffic cone', 'A tulip']                                     

                    INFO     Processing Task: DTD                                                        ]8;id=794913;file:///tmp/ipykernel_108035/4266128688.py\__main__]8;;\:]8;id=296004;file:///tmp/ipykernel_108035/4266128688.py#5\5]8;;\

2025-03-17 12:02:56 INFO     Task DTD: ['A photo taken at twilight', 'Cinematic portrait with dramatic  ]8;id=432617;file:///tmp/ipykernel_108035/4266128688.py\__main__]8;;\:]8;id=748679;file:///tmp/ipykernel_108035/4266128688.py#35\35]8;;\
                             lighting', 'A lattice design', 'Burst of colorful confetti', 'A polka                 
                             dot', 'Close-up of a textured bark', 'Artwork featuring barcode-like                  
                             lines', 'Image with a shattered mosaic pattern', 'Ephemeral soap bubble               
                             art', 'Delicate embroidery', 'captivating starry night', 'Artwork with                
                             Mondrian-like grids', 'A pine tree', 'Close-up of a textured reptile                  
                             skin', 'Urban labyrinth', 'Abstract artwork with splatter paint', 'Serene             
                             lakeside retreat', 'intricate crystal arrangement', 'Desert sand dunes',              
                             'Photograph taken in a toy store', 'Art Nouveau-inspired design', 'A                  
                             network of veins', 'Artwork featuring Morse code typography', 'Close-up of            
                             a textured seashore', 'Time-lapse trails', 'A skirt', 'Ocean sunset                   
                             silhouette', 'Picture taken in the Italian pasta kitchens', 'Image with               
                             harlequin patterns', 'Aerial view of a marketplace', 'A parabola', 'Image             
                             taken in the Florida Everglades', 'Image with a single road sign', 'Image             
                             with a sand dune landscape', 'Detailed illustration of a geological                   
                             formation', 'Tranquil waterfall scene', 'Tranquil intersections', 'Image              
                             with shattered glass art installation']                                               

                    INFO     Processing Task: EuroSAT                                                    ]8;id=539383;file:///tmp/ipykernel_108035/4266128688.py\__main__]8;;\:]8;id=715338;file:///tmp/ipykernel_108035/4266128688.py#5\5]8;;\

...

                    INFO     Task FashionMNIST: ['Sunlit foliage', 'Street art-inspired mural           ]8;id=756817;file:///tmp/ipykernel_108035/4266128688.py\__main__]8;;\:]8;id=186997;file:///tmp/ipykernel_108035/4266128688.py#35\35]8;;\
                             painting', 'A dress', 'Tranquil lakeside pier', 'Picture taken in the                 
                             Spanish Flamenco festivals', 'Aerial view of a inlet', 'Image with a                  
                             winding path or trail', 'Intense athlete', 'An image of Andorra', 'Artwork            
                             featuring Escher-like patterns', 'A boot', 'Artwork featuring zebra stripe            
                             motifs', 'Patchwork quilt design', 'Picture captured in the Icelandic                 
                             glaciers', 'A shirt', 'Picture taken in Seychelles', 'thrilling racing                
                             event', 'Image with vintage typography', 'Close-up of a textured silk',               
                             'Image with ornate arabesque patterns', 'Photo taken in Machu Picchu',                
                             'Time-lapse trails', 'intricate kaleidoscope pattern',                                
                             'Impressionist-style digital painting', 'A plank', 'Close-up of a textured            
                             synthetic wood', 'thrilling sports event', 'A bamboo', 'Minimalist                    
                             architectural photography', 'Image with tartan patterns', 'A scorpion',               
                             'Bursting fireworks display', 'A helix', 'An image of Luxembourg',                    
                             'Surrealist collage artwork', 'An image of Fiji', 'Image with camouflage              
                             print', 'A checkerboard pattern']                                                     

                    INFO     Processing Task: RenderedSST2                                               ]8;id=753126;file:///tmp/ipykernel_108035/4266128688.py\__main__]8;;\:]8;id=485;file:///tmp/ipykernel_108035/4266128688.py#5\5]8;;\

2025-03-17 12:03:00 INFO     Task RenderedSST2: ['Artwork featuring Morse code typography', 'Intricate  ]8;id=739467;file:///tmp/ipykernel_108035/4266128688.py\__main__]8;;\:]8;id=570759;file:///tmp/ipykernel_108035/4266128688.py#35\35]8;;\
                             pencil drawing', 'Photograph taken in a record shop', 'An image of                    
                             Luxembourg', 'Wide-angle perspective', 'Meticulously arranged flowerbed',             
                             'Photo taken in the Serengeti National Park', 'serene Japanese garden',               
                             'delicate soap bubble display', 'Nostalgic city pulse', 'An ellipse',                 
                             'Isolated subject', 'Intrica wood carvingte', 'Photo with muted,                      
                             desaturated tones', 'An image of Andorra', 'an image of liechtenstein', 'A            
                             stretcher', 'An image of cheeks', 'Patchwork quilt design', 'A geyser',               
                             'Vibrant humanity', 'A quasar', 'Photo taken in the Serengeti National                
                             Park', 'Ethereal quality', 'Intrica wood carvingte', 'Futuristic-edge                 
                             robotic innovation', 'Enchanting forest nymph aesthetic', 'Vivid cultural             
                             festival', 'Miniature diorama photography', 'Intrica cathedralte',                    
                             'Submerged underwater scene', 'A parallelogram', 'A vine', 'Facial                    
                             close-up', 'Tranquil oceanside retreat', 'A bonnet', 'A photo with the                
                             letter S', 'Plaid pattern']                                                           

                    INFO     Processing Task: KMNIST                                                     ]8;id=8054;file:///tmp/ipykernel_108035/4266128688.py\__main__]8;;\:]8;id=211949;file:///tmp/ipykernel_108035/4266128688.py#5\5]8;;\

                    INFO     Task KMNIST: ['Photograph taken in a car', 'Captivating street life',      ]8;id=538527;file:///tmp/ipykernel_108035/4266128688.py\__main__]8;;\:]8;id=968851;file:///tmp/ipykernel_108035/4266128688.py#35\35]8;;\
                             'Close-up of a textured bark', 'Image with a whirlpool in the ocean',                 
                             'Glowing neon cityscape', 'intricate clockwork gears', 'Quiet rural                   
                             farmhouse', 'Close-up of a textured synthetic leather', 'Illustration of a            
                             hidden ancient script', 'A seal', 'intricate gemstone arrangement',                   
                             'Ephemeral soap bubble creation', 'Vibrant floral arrangement', 'Arabic               
                             script calligraphy', 'Fast-paced race car blur', 'Artwork featuring 8-bit             
                             pixel art', 'Picture taken in Alberta, Canada', 'Blossoming springtime                
                             blooms', 'a picture of taiwan', 'detailed mosaic design', 'Close-up of a              
                             textured feather', 'Image captured in the Patagonian wilderness', 'Bokeh              
                             effect', 'Enigmatic silhouettes', 'Collage of vintage magazine clippings',            
                             'Aerial view of a snowy landscape', 'Photo taken in Monument Valley',                 
                             'Picture with a close-up of a flower', 'Remote alpine chalet', 'An image              
                             of a couple', 'Mesmerizing kinetic sculpture', 'Tranquil garden oasis',               
                             'Glistening dew-covered foliage', 'An image of Luxembourg', 'Arabic script            
                             calligraphy', 'Lone subject in vastness', 'Desert oasis palm trees',                  
                             'Graceful swimming fish']                                                             

In [26]:
output_path = Path(cfg.misc.output_dir) / f'nearest_neighbor_{cfg.model}.txt'

Path(cfg.misc.output_dir).mkdir(parents=True, exist_ok=True)
with open(output_path, "w") as f:
    f.write(output_str)